In [1]:
!pip install pyspark findspark
import findspark
findspark.init()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=9a509797415296d9d2a7d54ab60ad77bda4522be1f0f80f01685a8b4241de730
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [4]:
from pyspark.sql.functions import expr,col,isnan,when,count,regexp_replace,sum
from pyspark.sql.types import IntegerType,DoubleType,StringType,StructType,StructField,DateType
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("customer_product_project").getOrCreate()

In [5]:
from google.colab import files
uploaded = files.upload()


Saving sales.csv.txt to sales.csv.txt


In [7]:
file_name1 = list(uploaded.keys())[0]
schema = StructType(
    [StructField("product_id",IntegerType(), True),
     StructField("customer_id",StringType(), True),
     StructField("order_date",DateType(), True),
     StructField("location",StringType(), True),
     StructField("source_order",StringType(), True)
    ]
)
# Read the CSV file using PySpark
# playstore_df = spark.read.csv(file_name1, header=True, inferSchema=True)
sales_df = spark.read.format("csv").option("inferschema","True").schema(schema).load(file_name1)

# Display the data (use show() instead of display)
sales_df.show(2)

+----------+-----------+----------+--------+------------+
|product_id|customer_id|order_date|location|source_order|
+----------+-----------+----------+--------+------------+
|         1|          A|2023-01-01|   India|      Swiggy|
|         2|          A|2022-01-01|   India|      Swiggy|
+----------+-----------+----------+--------+------------+
only showing top 2 rows



In [9]:
from pyspark.sql.functions import month, year, quarter
sales_df=sales_df.withColumn("order_year",year(sales_df.order_date))\
         .withColumn("order_month",month(sales_df.order_date))\
         .withColumn("order_quarter",quarter(sales_df.order_date))
sales_df.show(2)

+----------+-----------+----------+--------+------------+----------+-----------+-------------+
|product_id|customer_id|order_date|location|source_order|order_year|order_month|order_quarter|
+----------+-----------+----------+--------+------------+----------+-----------+-------------+
|         1|          A|2023-01-01|   India|      Swiggy|      2023|          1|            1|
|         2|          A|2022-01-01|   India|      Swiggy|      2022|          1|            1|
+----------+-----------+----------+--------+------------+----------+-----------+-------------+
only showing top 2 rows



In [15]:
from google.colab import files
uploaded1 = files.upload()

Saving menu.csv.txt to menu.csv.txt


In [18]:
file_name1 = list(uploaded1.keys())[0]
schema1 = StructType(
    [StructField("product_id",IntegerType(), True),
     StructField("product_name",StringType(), True),
     StructField("price",StringType(), True)
    ]
)
# Read the CSV file using PySpark
# playstore_df = spark.read.csv(file_name1, header=True, inferSchema=True)
menu_df = spark.read.format("csv").option("inferschema","True").schema(schema1).load(file_name1)

# Display the data (use show() instead of display)
menu_df.show(2)
sales_df.show(2)

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         1|       PIZZA|  100|
|         2|     Chowmin|  150|
+----------+------------+-----+
only showing top 2 rows

+----------+-----------+----------+--------+------------+----------+-----------+-------------+
|product_id|customer_id|order_date|location|source_order|order_year|order_month|order_quarter|
+----------+-----------+----------+--------+------------+----------+-----------+-------------+
|         1|          A|2023-01-01|   India|      Swiggy|      2023|          1|            1|
|         2|          A|2022-01-01|   India|      Swiggy|      2022|          1|            1|
+----------+-----------+----------+--------+------------+----------+-----------+-------------+
only showing top 2 rows



In [19]:
# Apply Join on both
cust_prod_df = sales_df.join(menu_df,"product_id")
cust_prod_df.show(2)


+----------+-----------+----------+--------+------------+----------+-----------+-------------+------------+-----+
|product_id|customer_id|order_date|location|source_order|order_year|order_month|order_quarter|product_name|price|
+----------+-----------+----------+--------+------------+----------+-----------+-------------+------------+-----+
|         1|          A|2023-01-01|   India|      Swiggy|      2023|          1|            1|       PIZZA|  100|
|         2|          A|2022-01-01|   India|      Swiggy|      2022|          1|            1|     Chowmin|  150|
+----------+-----------+----------+--------+------------+----------+-----------+-------------+------------+-----+
only showing top 2 rows



In [23]:
# Total Amount spent by each customer
cust_prod_df.createOrReplaceTempView("cust_prod")
total_amount_df = spark.sql("select customer_id, sum(price) from cust_prod group by 1 order by 2 desc")
total_amount_df.show(5)

+-----------+----------+
|customer_id|sum(price)|
+-----------+----------+
|          B|    4440.0|
|          A|    4260.0|
|          C|    2400.0|
|          E|    2040.0|
|          D|    1200.0|
+-----------+----------+



In [28]:
from pyspark.sql.functions import expr,col,isnan,when,count,regexp_replace,sum,desc
total_amount_df = cust_prod_df.groupBy("customer_id").agg(sum("price").alias("total_amount"))
total_amount_df = total_amount_df.orderBy(desc("total_amount"))
total_amount_df.show(5)

+-----------+------------+
|customer_id|total_amount|
+-----------+------------+
|          B|      4440.0|
|          A|      4260.0|
|          C|      2400.0|
|          E|      2040.0|
|          D|      1200.0|
+-----------+------------+



In [29]:
# Total Amount spent by each food category
total_amount_df = cust_prod_df.groupBy("product_name").agg(sum("price").alias("total_amount"))
total_amount_df = total_amount_df.orderBy(desc("total_amount"))
total_amount_df.show(5)

+------------+------------+
|product_name|total_amount|
+------------+------------+
|    sandwich|      5760.0|
|     Chowmin|      3600.0|
|       PIZZA|      2100.0|
|        Dosa|      1320.0|
|       Pasta|      1080.0|
+------------+------------+
only showing top 5 rows



In [31]:
# Total Amount of sales in each month
total_amount_df = cust_prod_df.groupBy("order_month","order_year").agg(sum("price").alias("total_amount"))
total_amount_df = total_amount_df.orderBy(desc("total_amount"))
total_amount_df.show(5)

+-----------+----------+------------+
|order_month|order_year|total_amount|
+-----------+----------+------------+
|          5|      2023|      2240.0|
|          6|      2023|      2240.0|
|          1|      2023|      2240.0|
|          2|      2023|      1680.0|
|          2|      2022|      1050.0|
+-----------+----------+------------+
only showing top 5 rows



In [33]:
# Yearly sales
total_amount_df = cust_prod_df.groupBy("order_year").agg(sum("price").alias("total_amount"))
total_amount_df = total_amount_df.orderBy(desc("total_amount"))
total_amount_df.show(5)

+----------+------------+
|order_year|total_amount|
+----------+------------+
|      2023|      9990.0|
|      2022|      4350.0|
+----------+------------+



In [35]:
# Quarterly sales
total_amount_df = cust_prod_df.groupBy("order_year","order_month","order_quarter").agg(sum("price").alias("total_amount"))
total_amount_df = total_amount_df.orderBy(desc("total_amount"))
total_amount_df.show(5)

+----------+-----------+-------------+------------+
|order_year|order_month|order_quarter|total_amount|
+----------+-----------+-------------+------------+
|      2023|          1|            1|      2240.0|
|      2023|          5|            2|      2240.0|
|      2023|          6|            2|      2240.0|
|      2023|          2|            1|      1680.0|
|      2022|          2|            1|      1050.0|
+----------+-----------+-------------+------------+
only showing top 5 rows



In [39]:
# Total number of order by each category
total_order_df = cust_prod_df.groupBy("product_id","product_name").agg(count("product_id").alias("total_order"))
total_order_df = total_order_df.orderBy(desc("total_order"))
total_order_df.show(5)

+----------+------------+-----------+
|product_id|product_name|total_order|
+----------+------------+-----------+
|         3|    sandwich|         48|
|         2|     Chowmin|         24|
|         1|       PIZZA|         21|
|         4|        Dosa|         12|
|         5|     Biryani|          6|
+----------+------------+-----------+
only showing top 5 rows



In [40]:
# Top 5 ordered items
total_order_df = cust_prod_df.groupBy("product_id","product_name").agg(count("product_id").alias("total_order"))
total_order_df = total_order_df.orderBy(desc("total_order"))
total_order_df.show(5)

+----------+------------+-----------+
|product_id|product_name|total_order|
+----------+------------+-----------+
|         3|    sandwich|         48|
|         2|     Chowmin|         24|
|         1|       PIZZA|         21|
|         4|        Dosa|         12|
|         5|     Biryani|          6|
+----------+------------+-----------+
only showing top 5 rows



In [41]:
# Top most ordered item
total_order_df = cust_prod_df.groupBy("product_id","product_name").agg(count("product_id").alias("total_order"))
total_order_df = total_order_df.orderBy(desc("total_order"))
total_order_df.show(1)

+----------+------------+-----------+
|product_id|product_name|total_order|
+----------+------------+-----------+
|         3|    sandwich|         48|
+----------+------------+-----------+
only showing top 1 row



In [44]:
# Frequency of customer visited to source_order
total_visit_df = cust_prod_df.groupBy("customer_id","source_order").agg(count("customer_id").alias("total_visits"))
total_visit_df = total_visit_df.orderBy(desc("total_visits"))
total_visit_df.show(5)

+-----------+------------+------------+
|customer_id|source_order|total_visits|
+-----------+------------+------------+
|          A|      Swiggy|          21|
|          B|      Swiggy|          18|
|          B|      zomato|          12|
|          C|      zomato|           9|
|          A|  Restaurant|           9|
+-----------+------------+------------+
only showing top 5 rows



In [46]:
# Frequency of customer visited to Restaurant
from pyspark.sql.functions import expr,col,isnan,when,count,regexp_replace,sum,desc,countDistinct
total_visit_df = cust_prod_df.filter(col("source_order")=="Restaurant").groupBy("customer_id").agg(countDistinct("order_date").alias("total_visits"))
total_visit_df = total_visit_df.orderBy(desc("total_visits"))
total_visit_df.show(5)

+-----------+------------+
|customer_id|total_visits|
+-----------+------------+
|          B|           6|
|          A|           6|
|          E|           5|
|          C|           3|
|          D|           1|
+-----------+------------+



In [47]:
# Total sales by each country
total_amount_df = cust_prod_df.groupBy("location").agg(sum("price").alias("total_amount"))
total_amount_df = total_amount_df.orderBy(desc("total_amount"))
total_amount_df.show(5)

+--------+------------+
|location|total_amount|
+--------+------------+
|      UK|      7020.0|
|   India|      4860.0|
|     USA|      2460.0|
+--------+------------+



In [48]:
# Total sales by order source
total_amount_df = cust_prod_df.groupBy("source_order").agg(sum("price").alias("total_amount"))
total_amount_df = total_amount_df.orderBy(desc("total_amount"))
total_amount_df.show(5)

+------------+------------+
|source_order|total_amount|
+------------+------------+
|      Swiggy|      6330.0|
|      zomato|      4920.0|
|  Restaurant|      3090.0|
+------------+------------+



In [49]:
# Total sales by order source and country
total_amount_df = cust_prod_df.groupBy("source_order","location").agg(sum("price").alias("total_amount"))
total_amount_df = total_amount_df.orderBy(desc("total_amount"))
total_amount_df.show(5)

+------------+--------+------------+
|source_order|location|total_amount|
+------------+--------+------------+
|      Swiggy|      UK|      2910.0|
|      Swiggy|   India|      2760.0|
|  Restaurant|      UK|      2070.0|
|      zomato|      UK|      2040.0|
|      zomato|     USA|      1800.0|
+------------+--------+------------+
only showing top 5 rows

